In [1]:
### Data Cleaning

In [2]:
# importing libraries
import pandas as pd
import numpy as np

Reading dataset

In [3]:
data_uni = pd.read_excel("Data Uni Dissertation.xlsx")

data_uni

FileNotFoundError: [Errno 2] No such file or directory: 'Data Uni Dissertation.xlsx'

Exploring Data

In [ ]:
#generating profiling report

from pandas_profiling import ProfileReport
##report = ProfileReport(data_uni)
#report.to_file(output_file= "report.html")

Cleaning Data

In [ ]:
# Dealing with  inconsistent values

def trimmer(df):
    column_list = []

    for (columnName, columnData) in df.iteritems():
       
        if columnData.dtype == "object":

            df_replace = columnData.str.strip()

            column_list.append(df_replace)
        else:
            column_list.append(columnData)
            

    return (pd.DataFrame(column_list))

trimmer(data_uni)


In [ ]:
# creating a new df after trimming 
data_uni_df = trimmer(data_uni)
data_uni_df = data_uni_df.transpose( copy = True)

#saving the new data frame
data_uni_df.to_csv("data_uni.csv", index = False)
data_uni_2 = pd.read_csv("data_uni.csv")

imputting missing values

In [ ]:
#filling nan with zeros
data_uni_2.iloc[:, [24,25,29]]= data_uni_df.iloc[:, [24,25,29]].fillna(0)

data_uni_2.isna().sum()

In [ ]:
#imputing missing values with most frequent values
from sklearn.impute import SimpleImputer


imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(data_uni_2.iloc[:, [0,4,6,7,8,9,11,12,13,14,18,19,20,21]])
data_uni_2.iloc[:, [0,4,6,7,8,9,11,12,13,14,18,19,20,21]]= imputer.transform(data_uni_2.iloc[:, [0,4,6,7,8,9,11,12,13,14,18,19,20,21]])


In [ ]:
#imputing missing values with median values
data_num = data_uni_2.select_dtypes(include="number")

imputer_2 = SimpleImputer(missing_values=np.nan, strategy='median')
imputer_2.fit(data_num.iloc[:, [1,2,3]])
data_num.iloc[:, [1,2,3]] = imputer_2.transform(data_num.iloc[:, [1,2,3]])

data_uni_2.iloc[:, [15,16,17]] = data_num.iloc[:, [1,2,3]]

In [ ]:
#Generating report with clean data
#report2 = ProfileReport(data_uni_2)
#report2.to_file(output_file= "report2.html")

Dropping Highly Correlated and unseful Features

In [ ]:
data_uni_2= data_uni_2.drop(["Course","AWM year 1", "AWM year 2", "AWM year 3", "Overall AWM", "Attendance"], axis = 1)

Encoding Feautures

In [ ]:
#ethinicty binarisation 

data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("White", "0")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Asian", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Black/Black British African", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Other ethnic background", "1")

print(data_uni_2.Ethnicity)

In [ ]:
# Polar 4 Score Binarisation
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.str.replace("5", "1")
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.str.replace("4", "1")
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.str.replace("3", "1")
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.str.replace("2", "0")

print(data_uni_2.Polar_4_Score)


In [ ]:
#splitting X and y
y = data_uni_2["desertion"]
X = data_uni_2.drop(["desertion"], axis = 1)

In [ ]:
#Encoding dependent Variable
from sklearn.preprocessing import LabelEncoder
from collections import Counter

le = LabelEncoder()
y_enc = le.fit_transform(y)
Counter(y)

Test 1

In [ ]:
#Encoding independant variables

from sklearn.preprocessing import OneHotEncoder


X_cat = X.select_dtypes(exclude="number")

enc = OneHotEncoder(handle_unknown= "error", sparse = False, drop = "if_binary")

X_cat= np.array(enc.fit_transform(X_cat))

X_cat = pd.DataFrame(X_cat )


X_cat.columns = enc.get_feature_names()
print(X_cat)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_num = X.select_dtypes(include="number")
X_num_colnames= X_num.columns 
a = sc.fit_transform(X_num.to_numpy())
X_num = pd.DataFrame(a)
X_num.columns = X_num_colnames

X_num_cat = pd.concat((X_num, X_cat), axis =1)

print(X_num_cat)

Test 2

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

columns_enc = X.select_dtypes(include="number")
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse= False, drop = "if_binary", handle_unknown= "error"), columns_enc )], remainder='passthrough')

X_test_2 = np.array(ct.fit_transform(X))
X_test_2 = pd.DataFrame(X_test_2)
